<a href="https://colab.research.google.com/github/Goodman2442/test11/blob/main/%D0%A4%D0%B8%D1%88%D0%BA%D0%B8_GPT_%7C_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%7C_%D0%A3%D0%98%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning контурной моделии и подготовка набора данных.**

Практическую часть занятия проведем на примере того же синтетического датасета (сгенерированных данных) по нашей базе УИИ вопрос/ответ (без диалоговости), что и в предыдущей лекции. Таким образом исследуем поведение обучения на контурной модели.

Напомним:
**Датасет собран из  сгенеренных 1500 вопросов (по 5 вопросов на чанк)** по базе УИИ (gpt-3.5-turbo-0125).

Были **сгенерировны ответы от 2 моделей по чанкам (фрагментам текста) по которым генерились вопросы** (gpt-3.5-turbo-0125 и контурная лучшая из нашего списка stablebelsga 13b) и с промтом нашего нейро-мнеджера.

Таким образом **собралось 3000 ответов от 2 моделей**.

Затем **провели автоматическую оценку ответов (с помощью gpt-3.5-turbo-0125)** на соотвествие информации в чанках, и таким образом из ответов **gpt-3.5-turbo-0125 удалилось 30 примеровв а из ответов stablebeluga-13b около 500 примеров**.

Итоговый **датасет составил 2460 примеров покрывающих всю базу знаний УИИ.**


**Комментарий к выбору библиотеки Unsloth для fine-tuning:**

1. Весь код можно выполнить на бесплатном экземпляре Tesla T4 Google Colab!
2. Вы узнаете, как подготовить данные, как обучать, как запускать модель и как ее сохранять (например, для Llama.cpp).
3. Unsloth поддерживает Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes и т. д.
4. Unsloth поддерживает 16-битный LoRA или 4-битный QLoRA. Оба в 2 раза быстрее.
5. Для max_seq_length можно задать любое значение, поскольку Unsloth выполняет автоматическое масштабирование RoPE с помощью метода kaiokendev.
6. С версии PR 26037 Unsloth поддерживает загрузку 4-битных моделей в 4 раза быстрее! В Unsloth репо есть модели Llama, Mistral 4bit.

**Выбранная модель Llama-3 8b обучена на огромнейших 15 триллионах токенов! Тогда как llama-2 обучена на 2 триллионах.**

### Подключение Google Диска

Код монтирует Google Диск к виртуальной машине Google Colab, предоставляя доступ к файлам на диске.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Установка зависимостей для использования PyTorch и Unsloth

Этот код устанавливает необходимые пакеты для работы с PyTorch и библиотекой Unsloth в зависимости от версии GPU, доступной в среде Google Colab.

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

### Загрузка и настройка метода FastLanguageModel для модели llama-3-8b-Instruct

Этот код выполняет следующие действия:

1. Импортирует библиотеку `FastLanguageModel` из модуля `unsloth` и `torch`.
2. Устанавливает параметры:
   - `max_seq_length`: максимальная длина последовательности (8192).
   - `dtype`: тип данных для модели (автоматическое определение или вручную).
   - `load_in_4bit`: использование 4-битной квантизации для уменьшения использования памяти.
3. Определяет список предобученных моделей, поддерживающих 4-битную квантизацию.
4. Загружает выбранную модель и токенизатор с использованием заданных параметров.

Эти настройки позволяют эффективно использовать мощные модели с ограниченными ресурсами памяти и ускоряют процесс загрузки.

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 8192  # Выберите любое значение! unsloth автоматически поддерживаем масштабирование RoPE!
dtype = None  # None для автоматического определения. Float16 для Tesla T4, V100, Bfloat16 для Ampere+.
load_in_4bit = True  # Используйте 4-битную квантизацию для уменьшения использования памяти. Можно установить False.

# Предварительно квантизированные модели в 4-битном формате, которые unsloth поддерживает для 4x более быстрой загрузки и отсутствия ошибок переполнения памяти (OOM).
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Инструкционная версия Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Инструкционная версия Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit",  # [НОВОЕ] 15 триллионов токенов Llama-3
    "unsloth/llama-3-70b-Instruct-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit" # Инструкционная версия Llama-3
]  # Больше моделей на https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # используйте токен, если используете защищенные модели, такие как meta-llama/Llama-2-7b-hf
)


    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.2.1+cu121)
    Python  3.10.14 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA GeForce RTX 3060 Laptop GPU. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

### Настройка модели FastLanguageModel с использованием PEFT

Этот код выполняет следующие действия:

1. Настраивает модель `FastLanguageModel` с использованием PEFT (Parameter Efficient Fine-Tuning).
2. Параметры настройки:
   - `r=16`: задает значение ранга (рекомендуемые значения: 8, 16, 32, 64, 128).
   - `target_modules`: список модулей, на которые нацелена настройка.
   - `lora_alpha=16`: задает значение альфа-параметра LoRA.
   - `lora_dropout=0`: задает уровень dropout для LoRA (оптимально = 0).
   - `bias="none"`: задает значение смещения (оптимально = "none").
   - `use_gradient_checkpointing="unsloth"`: использует градиентное контрольное сохранение (оптимально для длинных контекстов).
   - `random_state=3407`: задает значение случайного состояния для воспроизводимости.
   - `use_rslora=False`: отключает использование Rank Stabilized LoRA.
   - `loftq_config=None`: отключает конфигурацию LoftQ.

Эта настройка оптимизирует использование памяти и ускоряет обучение модели, обеспечивая высокую производительность даже при ограниченных ресурсах.

Добавленный адаптер LoRA, позволяет обновить только от 1 до 10% всех параметров!

q_proj: Проекция запросов (Query Projection). Этот модуль отвечает за преобразование входных данных в запросы, которые используются в механизме внимания для поиска релевантной информации.

k_proj: Проекция ключей (Key Projection). Этот модуль преобразует входные данные в ключи, которые используются вместе с запросами для вычисления весов внимания.

v_proj: Проекция значений (Value Projection). Этот модуль преобразует входные данные в значения, которые комбинируются с весами внимания для получения окончательных результатов.

o_proj: Проекция выходов (Output Projection). Этот модуль отвечает за преобразование результатов механизма внимания в окончательные выходные данные.
gate_proj: Проекция гейтов (Gate Projection). Этот модуль используется для управления потоком информации через различные части модели, часто применяемый в механизмах, таких как LSTM или GRU.

up_proj: Проекция вверх (Up Projection). Этот модуль используется для увеличения размерности данных, что может быть полезно для улучшения представления информации.

down_proj: Проекция вниз (Down Projection). Этот модуль используется для уменьшения размерности данных, что помогает снизить вычислительные затраты и улучшить эффективность модели.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Выберите любое число > 0! Рекомендуемые значения: 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # Поддерживается любое значение, но = 0 оптимально
    bias="none",     # Поддерживается любое значение, но = "none" оптимально
    # [НОВОЕ] "unsloth" использует на 30% меньше видеопамяти, позволяет использовать в 2 раза большие размеры батчей!
    use_gradient_checkpointing="unsloth",  # True или "unsloth" для очень длинных контекстов
    random_state=3407,
    use_rslora=False,  # Мы поддерживаем Rank Stabilized LoRA
    loftq_config=None,  # И LoftQ
)


## Подготовка данных


### Загрузка файлов в Google Colab

Код предоставляет интерфейс для загрузки файлов с локального компьютера в среду Google Colab и выводит информацию о загруженных файлах, включая их названия и размеры.

In [ ]:
from google.colab import files

# Загрузка файлов
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving selected_qa_full.csv to selected_qa_full.csv
User uploaded file "selected_qa_full.csv" with length 17715490 bytes


### Загрузка и отображение данных из CSV файла в Google Colab

Этот код выполняет следующие действия:

1. Импортирует библиотеку `pandas`.
2. Загружает данные из CSV файла, название которого хранится в переменной `fn`.
3. Сохраняет загруженные данные в переменную `df` в виде DataFrame.
4. Отображает содержимое DataFrame `df`.

In [ ]:
import pandas as pd

df = pd.read_csv(fn)
df

,question,answer,context
0,Какие достижения университета Искусственного И...,Университет Искусственного Интеллекта (УИИ) яв...,# Описание УИИ\nУниверситет Искусственного Инт...
1,Какие достижения университета Искусственного И...,Основные достижения университета Искусственног...,# Описание УИИ\nУниверситет Искусственного Инт...
2,Какие языки обучения предоставляет УИИ на свои...,Университет Искусственного Интеллекта предоста...,# Описание УИИ\nУниверситет Искусственного Инт...
3,Какие языки обучения предоставляет УИИ на свои...,Университет Искусственного Интеллекта (УИИ) пр...,# Описание УИИ\nУниверситет Искусственного Инт...
4,Какие проекты были созданы студентами УИИ в об...,Студентами Университета Искусственного Интелле...,# Описание УИИ\nУниверситет Искусственного Инт...
...,...,...,...
2455,Какие возможности предоставляются студентам дл...,Студентам предоставляются следующие возможност...,Мы гарантируем трудоустройство своим студентам...
2456,Какие компании уже воспользовались услугами Ун...,Университет Искусственного интеллекта уже разр...,Мы гарантируем трудоустройство своим студентам...
2457,Какие компании уже воспользовались услугами Ун...,"Компании, которые уже воспользовались услугам...",Мы гарантируем трудоустройство своим студентам...
2458,Какие возможности предоставляются студентам дл...,Студентам предоставляются различные возможност...,Мы гарантируем трудоустройство своим студентам...


Этот код выполняет следующие действия:

1. Выводит значение в столбце `question` первой строки DataFrame `df`.
2. Выводит значение в столбце `answer` первой строки DataFrame `df`.

In [ ]:
print(df.iloc[0]['question'])
print(df.iloc[0]['answer'])

Какие достижения университета Искусственного Интеллекта в области образования?
Университет Искусственного Интеллекта (УИИ) является ведущим образовательным учреждением в России, специализирующимся исключительно на области искусственного интеллекта. УИИ признан лидером в образовании в сфере ИИ и является самым крупным университетом по ИИ в России и странах СНГ. Более 4700 студентов со всего мира выбрали УИИ для прохождения курса по AI, где они не только получают теоретические знания, но и активно применяют их в создании проектов по искусственному интеллекту. За время своего существования УИИ создал 372 проекта в области ИИ. Кроме того, Университет предоставляет консультации более чем 100 крупным и малым компаниям, помогая им внедрять искусственный интеллект в своей деятельности.


### Создание DataFrame для тонкой настройки модели

Этот код выполняет следующие действия:

1. Устанавливает системную инструкцию, которая будет использоваться для генерации ответов в стиле профессионального менеджера поддержки, продающего курсы по AI в Университете Искусственного интеллекта.
2. Создает пустой список `fine_tune_data` для хранения данных.
3. Проходит по всем строкам исходного DataFrame `df`, извлекая вопросы (`question`) и ответы (`answer`).
4. Добавляет каждую пару "вопрос-ответ" в список `fine_tune_data` в виде словаря с ключами: `instruction`, `input`, `output`.
5. Создает новый DataFrame `df_end`, содержащий три колонки: `instruction`, `input` и `output`, на основе списка `fine_tune_data`.

Таким образом, создается набор данных для дальнейшей тонкой настройки модели, чтобы она могла генерировать ответы по базе УИИ.

In [ ]:
system_instruction = """Вы профессиональный менеджер поддержки в чате компании (компания продает курсы по AI) Университет Искусственного интеллекта.
Ответьте на вопрос (запрос) так, чтобы человек захотел после ответа купить обучение. Отвечайте на русском языке!
"""
fine_tune_data = []

for index, row in df.iterrows():
  user = row['question']
  assistant = row['answer']
  fine_tune_data.append(
      {
          "instruction": system_instruction,
          "input": user,
          "output": assistant
  }
      )

df_end = pd.DataFrame(fine_tune_data, columns=["instruction", "input", "output"])
df_end

,instruction,input,output
0,Вы профессиональный менеджер поддержки в чате ...,Какие достижения университета Искусственного И...,Университет Искусственного Интеллекта (УИИ) яв...
1,Вы профессиональный менеджер поддержки в чате ...,Какие достижения университета Искусственного И...,Основные достижения университета Искусственног...
2,Вы профессиональный менеджер поддержки в чате ...,Какие языки обучения предоставляет УИИ на свои...,Университет Искусственного Интеллекта предоста...
3,Вы профессиональный менеджер поддержки в чате ...,Какие языки обучения предоставляет УИИ на свои...,Университет Искусственного Интеллекта (УИИ) пр...
4,Вы профессиональный менеджер поддержки в чате ...,Какие проекты были созданы студентами УИИ в об...,Студентами Университета Искусственного Интелле...
...,...,...,...
2455,Вы профессиональный менеджер поддержки в чате ...,Какие возможности предоставляются студентам дл...,Студентам предоставляются следующие возможност...
2456,Вы профессиональный менеджер поддержки в чате ...,Какие компании уже воспользовались услугами Ун...,Университет Искусственного интеллекта уже разр...
2457,Вы профессиональный менеджер поддержки в чате ...,Какие компании уже воспользовались услугами Ун...,"Компании, которые уже воспользовались услугам..."
2458,Вы профессиональный менеджер поддержки в чате ...,Какие возможности предоставляются студентам дл...,Студентам предоставляются различные возможност...


### Сохранение DataFrame в CSV файл

Этот код выполняет следующие действия:

1. Сохраняет созданный DataFrame `df_end` в CSV файл под названием "your_data_v2.csv".
2. Устанавливает параметр `index=False`, чтобы исключить индексы строк из сохраняемого файла.

В результате файл "your_data_v2.csv" будет содержать данные для тонкой настройки модели, организованные в три колонки: `instruction`, `input` и `output`.

In [ ]:
df_end.to_csv("your_data_v2.csv", index=False)

### Подготовка датасета для тонкой настройки модели

Этот код выполняет следующие действия:

1. Импортирует необходимые библиотеки `datasets` и `pandas`.
2. Определяет шаблон `llama3_prompt` для форматирования данных.
3. Загружает данные из CSV файла в DataFrame с помощью `pandas`.
4. Конвертирует DataFrame в формат `Dataset` библиотеки Hugging Face.
5. Определяет функцию `formatting_prompts_func` для форматирования данных согласно шаблону.
6. Применяет функцию форматирования к датасету.
7. Подготавливает форматированный датасет для дальнейшего использования в тонкой настройке модели.

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd

llama3_prompt = """<|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output}"""

# Загрузите ваш датасет, если он уже в формате Hugging Face Datasets
# Если у вас данные в CSV, сначала загрузите их в pandas DataFrame, а затем конвертируйте в Dataset
data = pd.read_csv("your_data_v2.csv")
formatted_dataset = Dataset.from_pandas(data)

# Форматирование датасета
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = llama3_prompt.format(system_prompt=instruction, prompt=input, output=output)
        texts.append(text)
    return { "text": texts}
pass

# Применяем форматирование
dataset = formatted_dataset.map(fchedormatting_prompts_func, bat=True)

# Теперь dataset готов к использованию для fine-tuning модели.

Map:   0%|          | 0/2460 [00:00<?, ? examples/s]

Этот код демонстрирует токенизацию первого текстового примера из форматированного датасета, используя токенизатор.

1. Извлекает первый текстовый пример из форматированного датасета.
2. Применяет токенизатор для преобразования текста в токены.
3. Возвращает токенизированный результат.

Этот шаг показывает как превращается текст в последовательность токенов, понятных модели.

In [ ]:
tokenizer(dataset[0]['text'])

{'input_ids': [128000, 128000, 128006, 9125, 128007, 271, 46710, 103665, 94520, 1840, 101811, 34613, 69844, 119138, 7753, 111434, 17165, 5927, 17756, 123763, 119743, 320, 101098, 8164, 39280, 89715, 28007, 105426, 57319, 18154, 15592, 8, 59842, 111034, 104583, 8341, 127681, 104996, 110285, 114799, 12394, 3114, 30656, 1506, 627, 60627, 48074, 109121, 13373, 106435, 320, 9136, 19619, 42057, 8, 50945, 11, 82250, 105627, 103365, 13337, 12394, 102948, 93747, 1506, 107185, 18264, 122512, 17618, 13, 104444, 5591, 56857, 106606, 13373, 110950, 106292, 110424, 53671, 4999, 128009, 128006, 882, 128007, 271, 106523, 50306, 111226, 106990, 121552, 102511, 127681, 104996, 110285, 107318, 1830, 12394, 3114, 30656, 1506, 5927, 104952, 115812, 30, 128009, 128006, 78191, 128007, 271, 41682, 111034, 104583, 8341, 127681, 104996, 110285, 107318, 1830, 12394, 3114, 30656, 1506, 320, 41682, 31274, 31274, 8, 103962, 104885, 43293, 16494, 102294, 112520, 100717, 121873, 102345, 5927, 108190, 11, 104073, 7233


## Обучение модели

**Текущие настройки**

**per_device_train_batch_size**: размер батча для одного устройства (**16**).
**gradient_accumulation_steps**: количество шагов для накопления градиентов (**4**)
**были выбраны для видеокарты A100**.

**Для T4 рекомендуем:**

**per_device_train_batch_size = 2**

**gradient_accumulation_steps = 4**

### Настройка тренера для тонкой настройки модели

Этот код выполняет следующие действия:

1. Импортирует необходимые классы `SFTTrainer` из `trl` и `TrainingArguments` из `transformers`.
2. Создает экземпляр `SFTTrainer` для обучения модели.

Параметры настройки:
- `model`: модель, которую необходимо обучить.
- `tokenizer`: токенизатор, связанный с моделью.
- `train_dataset`: датасет для обучения.
- `dataset_text_field`: поле текста в датасете.
- `max_seq_length`: максимальная длина последовательности.
- `dataset_num_proc`: количество процессов для обработки данных.
- `packing`: упаковка данных (False для коротких последовательностей).

Параметры обучения (`TrainingArguments`):
- `per_device_train_batch_size`: размер батча для одного устройства (16).
- `gradient_accumulation_steps`: количество шагов для накопления градиентов (4).
- `warmup_steps`: количество шагов для разогрева (5).
- `num_train_epochs`: количество эпох обучения (10).
- `learning_rate`: скорость обучения (2e-4).
- `fp16`: использование 16-битных вычислений (если bf16 не поддерживается).
- `bf16`: использование bfloat16 вычислений (если поддерживается).
- `logging_steps`: шаги логгирования (1).
- `optim`: оптимизатор (adamw_8bit).
- `weight_decay`: коэффициент убывания весов (0.01).
- `lr_scheduler_type`: тип планировщика скорости обучения (linear).
- `seed`: начальное значение для генератора случайных чисел (3407).
- `output_dir`: директория для сохранения результатов (outputs).


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=10,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/2460 [00:00<?, ? examples/s]

In [ ]:
#@title Показать текущую статистику памяти
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
5.605 GB of memory reserved.


Запуск обучения

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,460 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 380
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.897200
2,2.938500
3,2.796200
4,2.780500
5,2.406300
6,2.272100
7,1.937300
8,1.684400
9,1.445000
10,1.327600


Step,Training Loss
1,2.897200
2,2.938500
3,2.796200
4,2.780500
5,2.406300
6,2.272100
7,1.937300
8,1.684400
9,1.445000
10,1.327600


In [ ]:
#@title Показать итоговую статистику памяти и времени
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3672.7998 seconds used for training.
61.21 minutes used for training.
Peak reserved memory = 32.244 GB.
Peak reserved memory for training = 26.639 GB.
Peak reserved memory % of max memory = 81.498 %.
Peak reserved memory for training % of max memory = 67.331 %.


## Предсказание (инференс)

### Генерация ответа с использованием модели FastLanguageModel

Этот код выполняет следующие действия:

1. Устанавливает системную инструкцию для создания ответов в стиле профессионального менеджера поддержки, продающего курсы по AI.
2. Используем шаблон `llama3_prompt` для форматирования данных.
3. Включает нативный режим быстрого вывода для модели с помощью `FastLanguageModel.for_inference`.
4. Токенизирует текст, включающий системную инструкцию и пример входных данных, оставляя поле вывода пустым для генерации.
5. Перемещает токенизованные данные на устройство `cuda` для вычислений.
6. Генерирует ответ модели с использованием метода `generate`, задавая максимальное количество новых токенов и включив использование кеша.
7. Декодирует сгенерированные токены в текстовый формат и сохраняет результат в переменной `completion`.

In [ ]:
system_instruction = """Вы профессиональный менеджер поддержки в чате компании (компания продает курсы по AI) Университет Искусственного интеллекта.
Ответьте на вопрос (запрос) так, чтобы человек захотел после ответа купить обучение. Отвечайте на русском языке!
"""

FastLanguageModel.for_inference(model) # Включите встроенный вывод (в 2 раза быстрее)
inputs = tokenizer(
[
    llama3_prompt.format(
        system_prompt=system_instruction, # instruction
        prompt="Продолжи последовательность Фибоначи: 1, 1, 2, 3, 5, 8", # input
        output="", # output - оставьте это поле пустым для генерации!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
completion = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Пример генерации общей задачи** не относящейся к дататасету (показательно что **модель не утрачивает способности** выполнять другие задачи)

Этот код выполняет следующие действия:

1. Извлекает первый элемент из списка сгенерированных ответов `completion`.
2. Разделяет текст по строке 'assistant\n\n' и берет последнюю часть, содержащую сгенерированный ответ.
3. Удаляет лишние пробелы из сгенерированного текста.

Результат:
- Получение чистого текста сгенерированного ответа без лишних метаданных.


In [ ]:
completion[0].split('<|start_header_id|>assistant<|end_header_id|>\n\n')[-1].replace('<|eot_id|>', '')

'13'

Пример вопроса по теме из датасета (модель справляется, предоставляя достоверные данные)




In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    llama3_prompt.format(
        system_prompt=system_instruction, # instruction
        prompt="расскажи про направление ChatGPT: состав курса и его стоимость", # input
        output="", # output
    )
    ], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nВы профессиональный менеджер поддержки в чате компании (компания продает курсы по AI) Университет Искусственного интеллекта.\nОтветьте на вопрос (запрос) так, чтобы человек захотел после ответа купить обучение. Отвечайте на русском языке!\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nрасскажи про направление ChatGPT: состав курса и его стоимость<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nНаправление ChatGPT включает в себя следующие курсы:\n1. Основы Python\n2. Функции и работа с текстами\n3. Библиотека Pandas\n4. Основы работы с chatGPT. промт-инжениринг\n5. Embedding представление текстов. Алгоритм LangChain\n6. Создание баз знаний. Архитектура базы. Работа с копирайтерами\n7. Создание диалога с chatGPT. Соединение нескольких chatGPT моделей\n8. Нейро-ассистент на основе парсинга сайта. Система с несколькими базами знаний

Вы также можете использовать TextStreamer для непрерывного вывода - чтобы вы могли видеть генерацию токена за токеном, а не ждать все время!

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
        llama3_prompt.format(
        system_prompt=system_instruction, # instruction
        prompt="Какие возможности предоставляются студентам для удовлетворения общего интереса к AI и нейросетям?", # input
        output="", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Вы профессиональный менеджер поддержки в чате компании (компания продает курсы по AI) Университет Искусственного интеллекта.
Ответьте на вопрос (запрос) так, чтобы человек захотел после ответа купить обучение. Отвечайте на русском языке!
<|eot_id|><|start_header_id|>user<|end_header_id|>

Какие возможности предоставляются студентам для удовлетворения общего интереса к AI и нейросетям?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Студентам предоставляются следующие возможности для удовлетворения общего интереса к AI и нейросетям:
1. Участие в стажировках, где студенты могут создавать реальные проекты в команде с тимлидом.
2. Возможность участвовать в хакатонах и AI-соревнованиях.
3. Участие в создании реального проекта для компании.
4. Возможность стать AI-разработчиком и работать на заказ.
5. Возможность создания собственного проекта на полно

Еще пример вопроса по теме

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    llama3_prompt.format(
        system_prompt=system_instruction, # instruction
        prompt="можно описать последовательность действий для создания контента из моей CRM с использованием Google Colab?", # input
        output="", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Вы профессиональный менеджер поддержки в чате компании (компания продает курсы по AI) Университет Искусственного интеллекта.
Ответьте на вопрос (запрос) так, чтобы человек захотел после ответа купить обучение. Отвечайте на русском языке!
<|eot_id|><|start_header_id|>user<|end_header_id|>

можно описать последовательность действий для создания контента из моей CRM с использованием Google Colab?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Для создания контента из вашей CRM с использованием Google Colab можно выполнить следующие действия:

1. Зарегистрироваться на сайте [сайт] и авторизоваться.
2. Нажать на кнопку "Новый журнал йоги" для создания нового документа Google Colab.
3. Написать код в блокноте Google Colab, используя язык программирования Python.
4. С помощью кода подключиться к вашей CRM для извлечения данных.
5. Преобразовать эти да

## Тестирование на подборке вопросов (мы использовали две подборки по 50 вопросов 1) из датасета, 2) По теме но не пересекающимися из датасета) с сохранением результатов в папку проекта на гугл диске /content/drive/MyDrive/Проекты/Train_dataset/

In [ ]:
import pandas as pd

model_name = 'llama-3-8b-Instruct-bnb-4bit'

temperature = 0

file_path = "/content/Вопросы для тестирования 2 из датасета.csv" #@param {type:"string"}
file_name = file_path.split('/')[-1].split('.')[0]
df = pd.read_csv(file_path)
df = df[~df['Вопрос'].isnull()]

try:
  df2 = pd.read_csv('/content/gpt-3.5-turbo-0125_qa_full.csv')
except KeyError:
  df2 = None

new_data = []

for index, row in df.iterrows():
  question = row['Вопрос']

  try:
    context = row['Чанки']
  except KeyError:
    context = None

  try:
    answer_base = df2[df2['question'] == question]['answer'].values[0]
    context = df2[df2['question'] == question]['chunks'].values[0]
  except IndexError:
    answer_base = None

  system_instruction = """Вы профессиональный менеджер поддержки в чате компании (компания продает курсы по AI) Университет Искусственного интеллекта.
Ответьте на вопрос (запрос) так, чтобы человек захотел после ответа купить обучение. Отвечайте на русском языке!
"""
  # alpaca_prompt = Copied from above
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
      llama3_prompt.format(
          system_prompt=system_instruction, # instruction
          prompt=question, # input
          output="", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
  completion = tokenizer.batch_decode(outputs)

  new_data.append({'Вопрос': row['Вопрос'],
                   "Ответ из базы": answer_base,
                   'Ответ': completion[0].split('<|start_header_id|>assistant<|end_header_id|>\n\n')[-1].replace('<|eot_id|>', ''),
                   'Контекст': context,
                  "Оценка": None,
                  "Комментарий": None})
  print(completion[0].split('<|start_header_id|>assistant<|end_header_id|>\n\n')[-1].replace('<|eot_id|>', ''))

new_df = pd.DataFrame(new_data)
new_df.to_csv(f'/content/drive/MyDrive/Проекты/Train_dataset/result_{model_name}_{file_name}.csv', index=False)
new_df.to_excel(f'/content/drive/MyDrive/Проекты/Train_dataset/result_{model_name}_{file_name}.xlsx', index=False)
print("Завершено!")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


В Университете Искусственного Интеллекта (УИИ) доступны два тарифа для прохождения курсов: "Базовый" и "Основной". Основное различие между ними заключается в том, что тариф "Базовый" стоит 89 900 рублей, а тариф "Основной" - 109 900 рублей. Тариф "Основной" включает в себя дополнительные курсы, такие как "Python для анализа данных", "AI-директор" и "Timely DJ", а также гарантию трудоустройства.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Для прохождения курсов в Университете Искусственного интеллекта необходим нулевой уровень подготовки. Все курсы УИИ рассчитаны на людей без математических знаний и опыта программирования, поэтому отсутствие специальных знаний не станет препятствием для успешного прохождения программы.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Углубленный курс по текстам (NLP) включает в себя следующие темы:
1. Основы и особенности работы с нейросетями в сфере NLP.
2. Устройство и принципы работы нейросетей.
3. Решение широкого спектра задач, начиная от классификации текста, заканчивая NER и переводом предложений.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Стоимость курса по PyTorch составляет 39 900 рублей, а продолжительность прохождения курса составляет 10 недель.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Темы занятий, включенные в курс по PyTorch:
1. Мотивационная лекция с обзором возможностей PyTorch.
2. Тензоры 1: numpy vs pytorch.
3. Тензоры 2.
4. Сложные задачи. Оптимизация. Обратное распространение ошибки.
5. Сверточные слои.
6. Датасеты.
7. DCGAN, AE, VAE, RL, GenGAN, FCN, Segnet.
8. Нейронные сети для текстов. Embeddings.
9. Нейронные сети для текстов. Embeddings 2.
10. Работа с аудио.
11. Работа с генеративными сетями.
12. Работа с генеративными сетями 2.
13. Обработка аудио и речи.
14. Обработка аудио и речи 2.
15. Обработка текстов.
16. Обработка текстов 2.
17. Обработка табличных данных.
18. Обработка табличных данных 2.
19. Обработка табличных данных 3.
20. Обработка табличных данных 4.
21. Обработка табличных данных 5.
22. Обработка табличных данных 6.
23. Обработка табличных данных 7.
24. Обработка табличных данных 8.
25. Обработка табличных данных 9.
26. Обработка табличных данных 10.
27. Обработка табличных данных 11.
28. Обработка табличных данных 12.
29. Обработка таб

In [ ]:
new_df.head()

,Вопрос,Ответ из базы,Ответ,Контекст,Оценка,Комментарий
0,Какие тарифы доступны для прохождения курсов в...,В Университете Искусственного Интеллекта предо...,В Университете Искусственного Интеллекта (УИИ)...,# Прохождение курсов\n\n## Трифы УИИ\nВ УИИ су...,None,None
1,Какой уровень подготовки необходим для прохожд...,Для прохождения курсов в Университете Искусств...,Для прохождения курсов в Университете Искусств...,## Проверка домашних заданий студентов УИИ:\nП...,None,None
2,Какие требования к оборудованию и доступу в ин...,Для прохождения курсов в Университете Искусств...,Для прохождения курсов в Университете Искусств...,## Формат курсов\nВесь процесс программы курсо...,None,None
3,Какой документ получит студент после окончания...,После завершения программы курса и сдачи экзам...,После успешного завершения программы курса и с...,"## Документ, который получит студент после око...",None,None
4,Какие тарифы предлагаются для курса CV-разрабо...,Для курса CV-разработчик в Университете Искусс...,В Университете Искусственного Интеллекта предл...,"## Selection models, модели отбора.\nСпециалис...",None,None


## Сохранение, загрузка настроенных моделей

Чтобы сохранить окончательную модель в виде адаптеров LoRA, используйте либо push_to_hub от Huggingface для онлайн-сохранения, либо save_pretrained для локального сохранения.

**[ПРИМЕЧАНИЕ]** Это сохраняет ТОЛЬКО адаптеры LoRA, а не полную модель. Чтобы сохранить в 16-битном формате или GGUF, покажем далее!

### Сохранение настроенной модели

Этот код выполняет следующие действия:

1. Сохраняет настроенную модель локально в директорию "lora_model".
2. (Закомментировано) Опционально, может загрузить модель на онлайн хаб, если раскомментировать строку и указать имя модели и токен.

Локальное сохранение позволяет сохранить изменения и использовать модель на локальной машине, в то время как онлайн сохранение делает модель доступной для других через Huggingface hub.


In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

**Архивирование настроенной модели**

Этот код выполняет следующие действия:

- Создает zip-архив `lora_model_llama3_instr.zip` из директории `/content/lora_model`.

Архивирование позволяет легко передавать и хранить настроенную модель.


In [ ]:
!zip -r lora_model_llama3_instr.zip '/content/lora_model'

  adding: content/lora_model/ (stored 0%)
  adding: content/lora_model/README.md (deflated 66%)
  adding: content/lora_model/adapter_model.safetensors (deflated 7%)
  adding: content/lora_model/adapter_config.json (deflated 54%)


**Копирование архива модели на Google Диск**

Этот код выполняет следующие действия:

- Копирует zip-архив `lora_model_llama3_instr.zip` из текущей директории в указанное место на Google Диске.

Это позволяет сохранить архив модели на вашем Google Диске для дальнейшего использования и хранения.


In [ ]:
!cp /content/lora_model_llama3_instr.zip /content/drive/MyDrive/Проекты/Train_llms/lora_model_llama3_8b_instr_23_05_2024.zip

### Загрузка обученной модели и генерация ответа

Теперь, если вы хотите загрузить адаптеры LoRA, которые мы только что сохранили для вывода, установите для параметра «False» значение «True»:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # ВАША МОДЕЛЬ, ИСПОЛЬЗОВАННАЯ ДЛЯ ОБУЧЕНИЯ
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    llama3_prompt.format(
        system_prompt=system_instruction, # instruction
        prompt="можно описать последовательность действий для создания контента из моей CRM с использованием Google Colab?", # input
        output="", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

### Загрузка обученной модели и генерация ответа через библиотеку transformers Hugging Face
Вы также можете использовать AutoModelForPeftCausalLM Hugging Face.

**!!! Используйте это только в том случае,** если у вас не установлен unsloth. Это может быть безнадежно медленно, поскольку загрузка 4-битной модели не поддерживается, а **вывод Unsloth в 2 раза быстрее**.

In [ ]:
if False:
    # Крайне НЕ советем - используйте Unsloth, если возможно
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Сохранение в float16 для VLLM

Unsloth также поддерживает непосредственное сохранение в float16. Выберите «merged_16bit» для float16 или «merged_4bit» для int4.

Unsloth также разрешает использование адаптеров lora в качестве запасного варианта.

Используйте push_to_hub_merged для загрузки в свою учетную запись Hugging Face! Вы можете перейти на https://huggingface.co/settings/tokens и получить свои личные токены.

Этот код предоставляет варианты сохранения и загрузки обученной модели с использованием различных методов сохранения: слияние до 16-битного и 4-битного форматов, а также сохранение только адаптеров LoRA.

1. **Слияние до 16-битного формата и сохранение:**
   - Локальное сохранение модели и токенизатора с использованием метода `merged_16bit`.
   - Загрузка модели на хаб с использованием метода `merged_16bit` (с токеном аутентификации).

2. **Слияние до 4-битного формата и сохранение:**
   - Локальное сохранение модели и токенизатора с использованием метода `merged_4bit`.
   - Загрузка модели на хаб с использованием метода `merged_4bit` (с токеном аутентификации).

3. **Сохранение только адаптеров LoRA:**
   - Локальное сохранение адаптеров LoRA и токенизатора.
   - Загрузка адаптеров LoRA на Hugging Face hub (с токеном аутентификации).

Этот код позволяет гибко управлять сохранением и загрузкой модели в зависимости от потребностей и доступных ресурсов.

Установите для необходимого метода параметр «False» в значение «True»:

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### Преобразование GGUF/llama.cpp
Чтобы сохранить файл в `GGUF`/`llama.cpp`, Unsloth теперь поддерживает его изначально! Unsloth клонирует `llama.cpp` и по умолчанию сохраняем его как `q8_0`. Разрешается так же все методы, такие как `q4_k_m`. Используйте save_pretrained_gguf для локального сохранения и push_to_hub_gguf для загрузки в HF.

Некоторые поддерживаемые методы количественного анализа (полный список на Unsloth [странице Wiki](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Быстрое преобразование. Высокое потребление ресурсов, но в целом приемлемо.
* `q4_k_m` — рекомендуется. Использует Q6_K для половины тензоров focus.wv и feed_forward.w2, в противном случае — Q4_K.
* `q5_k_m` — рекомендуется. Использует Q6_K для половины тензоров focus.wv и feed_forward.w2, в противном случае — Q5_K.

Установите для необходимого метода параметр «False» в значение «True»:

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

## Вывод

Этим исследованием предполагались те же пункты что и в предыдущей лекции:  
1.   **сократить затраты на инструкцию** (в fine-tuning варианте промпт составляет 150 токенов)
2.   **обеспечить качественный вывод ответов** касающийся знаний по нашей базе
3.   **сократить время вывода ответа** уменьшением промта, и контекста (так как есть прямая зависимость)

**По тестовым вопросам можно судить что новые знания у модели появились, в значительно большей степени чем при обучении chatgpt.**

По обучения мы протестировали модель **на 50 вопросах из датасета** и  получили общую оценку **0.38 (по 10 бальной шкале 6)** (по шкале от -2 до 2) и **на 50 общих вопросах по базе**  но не присутствующих или с измененным смыслом вопросов (не из датасета а которые реально задают пользователи) и  получили общую оценку **-0,06 (по 10 бальной шкале 4.9)** (по шкале от -2 до 2).

**Напомним** что у текущего нейросторудника с RAG нструкция (оценка **1,82**) на текущий момент составляет ~1500 токенов, а контекст для ответа (RAG) в диапозоне 3000-5000 токенов.

**Общие выводы:**

*   Библиотека Unsloth имеет богатый функционал, низкие затраты ресурсов на обучение, высокую скорость и достаточно простой код использования.

*   Исследование показало, что fine-tuning не всегда обеспечивает полное усвоение специфических данных (например, фамилий, названий курсов и цен), однако позволяет модели придерживаться стиля ответа и воспроизводить общую структуру.
*   Модель показала способность генерировать ответы с новой информацией и улучшила свои навыки адаптации к реальным вопросам пользователей.
*   Сокращение инструкций и времени вывода ответа являются значительными положительными достижениями, которые могут быть полезны в дальнейшем развитии и использовании модели.

В целом, проведенное исследование показало **положительные результаты** но **не заменяющим RAG (Retrieval Augmented Generation)**, которые могут быть использованы для дальнейшего совершенствования и оптимизации AI-моделей.